In [1]:
import pandas as pd
import numpy as np
from numpy import dot
from numpy.linalg import norm
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
import seaborn as sns
from functools import reduce
from scipy.spatial import distance
import pathlib
import os
from sklearn.decomposition import PCA
import json
import random
from random import sample

random.seed(30)

In [2]:
# Set data input folder
input_folder = "inputs"

# Set output folder, subfolder
output_folder = "outputs"
if not os.path.exists(output_folder):
    os.makedirs(output_folder, exist_ok=True)

In [3]:
# Import the guide-level profiles
df_guide = pd.read_csv("outputs/20240202_6W_CP498_SABER_Pilot_HeLa_SABER_only_guide_normalized_merged_feature_select_median_ALLWELLS_1_2.csv.gz")
# Subset the nontargeting guide profiles 
df_nontargeting = df_guide.query("Metadata_Foci_Barcode_MatchedTo_GeneCode == 'nontargeting'")

# Load hits from the hit calling process
whole_cell_hits = pd.read_csv('outputs/HeLa_SABER_plate_level_median_per_feat_sig_genes_1_FDR_whole_cell_hits.csv')
comp_spec_hits = pd.read_csv('outputs/HeLa_SABER_plate_level_median_per_feat_sig_genes_1_FDR_compartment_specific_hits.csv')
all_hits = pd.concat([whole_cell_hits,comp_spec_hits])
hit_list = list(comp_spec_hits.Gene) + list(whole_cell_hits.Gene)
whole_cell_hit_list = list(whole_cell_hits.Gene)
comp_spec_hit_list = list(comp_spec_hits.Gene)

# list non hit genes
all_genes_list = list(df_guide.Metadata_Foci_Barcode_MatchedTo_GeneCode.unique())
all_genes_list.remove("nontargeting")
non_hit_list = [gene for gene in all_genes_list if gene not in hit_list]
print("All genes:",len(all_genes_list),"non-hit genes",len(non_hit_list),'Whole cell hits',len(whole_cell_hit_list),'Compartment hits',len(comp_spec_hit_list))


All genes: 590 non-hit genes 202 Whole cell hits 294 Compartment hits 94


In [4]:
df_temp = df_guide.copy(deep=True)
features = list(df_guide.columns)
gene_list = list(df_temp.Metadata_Foci_Barcode_MatchedTo_GeneCode)
df_temp = df_temp.drop('Metadata_Foci_Barcode_MatchedTo_GeneCode',axis=1).set_index('Metadata_Foci_Barcode_MatchedTo_Barcode')
df_temp = df_temp.reset_index()
df_temp["Metadata_Foci_Barcode_MatchedTo_GeneCode"] = gene_list
df_temp = df_temp[features]
df_temp

,Metadata_Foci_Barcode_MatchedTo_GeneCode,Metadata_Foci_Barcode_MatchedTo_Barcode,Cells_AreaShape_BoundingBoxMaximum_Y,Cells_AreaShape_CentralMoment_0_1,Cells_AreaShape_CentralMoment_0_3,Cells_AreaShape_CentralMoment_1_0,Cells_AreaShape_CentralMoment_1_1,Cells_AreaShape_CentralMoment_1_2,Cells_AreaShape_CentralMoment_1_3,Cells_AreaShape_CentralMoment_2_1,...,Nuclei_Texture_SumEntropy_Syto9_10_01_256,Nuclei_Texture_SumEntropy_Syto9_10_03_256,Nuclei_Texture_SumEntropy_btubulin_10_01_256,Nuclei_Texture_SumVariance_NfKb_10_01_256,Nuclei_Texture_SumVariance_Syto9_10_00_256,Nuclei_Texture_SumVariance_Syto9_10_02_256,Nuclei_Texture_SumVariance_TDP43_10_01_256,Nuclei_Texture_SumVariance_TDP43_10_02_256,Nuclei_Texture_SumVariance_TDP43_10_03_256,Nuclei_Texture_Variance_Syto9_10_01_256
0,AARS2,AAAGGCGGCCCTCACGGCCG,-0.158375,0.013107,-0.100975,-0.163630,0.189520,0.285264,0.125879,-0.278317,...,0.091347,-0.301305,-0.026015,-0.436075,-0.328195,-0.294850,0.181285,0.130507,0.198450,-0.292855
1,AARS2,AGCAAACTGGGGTCGCCGCG,-0.459740,-0.272072,-0.060422,-0.397970,0.081770,-0.033692,0.056625,0.040295,...,-0.636985,-0.491365,-0.659530,-0.226015,-0.372045,-0.458385,-0.075922,0.011385,0.051656,-0.447315
2,AARS2,CCAACTTCTACGCAGAACAG,-0.252800,0.099935,0.130246,0.048945,-0.025845,-0.033111,0.097072,0.127338,...,-0.164985,-0.579905,-0.419715,-0.236695,-0.223783,-0.305315,0.158725,0.008738,0.000045,-0.240930
3,AARS2,GCTGAGCCAGTTCAGAAGCA,0.187520,0.034279,-0.255330,0.678388,-0.058525,-0.052645,-0.016378,0.434255,...,-0.584245,-0.199930,-0.628520,-0.422055,-0.499170,-0.543055,-0.082475,0.050340,0.046263,-0.523410
4,AARSD1,ACCTCCGCTCCCAATCTACC,-0.368480,-0.168987,0.040285,-0.057675,0.577885,0.510590,0.619805,-0.120377,...,-0.302223,-0.472220,0.350025,-0.310835,-0.825055,-0.658360,-0.260640,-0.189415,0.186295,-0.276285
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2395,nontargeting,TAAGATCCGCGGGTGGCAAC,0.178550,-0.198601,0.273680,0.202490,-1.244100,-0.728195,-0.646360,-0.909224,...,0.694340,0.549655,0.474175,0.154985,0.141940,0.532435,0.557760,0.352925,0.373555,0.429580
2396,nontargeting,TCCCGGTTGGTGAACGATAC,-0.608585,-0.530960,-0.001097,-0.284394,-0.058880,0.266767,0.231285,0.328587,...,-0.356770,-0.362107,-0.478795,-0.314249,-0.301765,-0.306024,0.206455,0.034544,0.052445,-0.406645
2397,nontargeting,TGCCGTGAAAAGACGCTGCG,-0.475410,-0.097035,0.043919,-0.068639,0.464080,0.092188,0.296787,-0.118563,...,-0.301825,-0.435390,-0.297345,-0.375630,-0.512320,-0.335140,0.150539,-0.028893,0.091351,-0.527665
2398,nontargeting,TGGCCACGAATTCCGCCGCC,0.366528,-0.257825,-0.370588,-0.140465,0.078908,0.190906,0.201455,0.021170,...,-0.188425,-0.329340,-0.501910,-0.291190,0.042340,-0.101190,0.230875,0.110184,0.282550,-0.207045


In [5]:
df_temp = df_guide.drop('Metadata_Foci_Barcode_MatchedTo_GeneCode',axis=1).set_index('Metadata_Foci_Barcode_MatchedTo_Barcode')
# Perform principal component analysis on hit list
pca = PCA()
pca.fit(df_temp)
x = list(pca.explained_variance_ratio_)
# Find principal component that represents 90% variation
PCA_lookup = {}
for i in range(len(x)):
    distance = abs(.9-sum(x[:i+1]))
    PCA_lookup[distance] = i 
component = PCA_lookup[min(PCA_lookup.keys())]+1
print (f'Principal component representing closest to 90% variation is {component}')
# Perform principal component analysis and select components representing 90% of variation in data
pca = PCA(n_components=component)
df_guide_pca = pd.DataFrame(pca.fit_transform(df_temp),index=df_temp.index)
df_guide_pca.head()

Principal component representing closest to 90% variation is 127


,0,1,2,3,4,5,6,7,8,9,...,117,118,119,120,121,122,123,124,125,126
Metadata_Foci_Barcode_MatchedTo_Barcode,,,,,,,,,,,,,,,,,,,,,
AAAGGCGGCCCTCACGGCCG,-5.785421,-0.378360,11.583177,1.795391,-4.140418,1.017659,-0.860803,-1.168504,0.559236,1.871858,...,0.221074,-0.369346,-0.458958,0.122728,0.344767,0.680401,-0.462260,-0.009803,-0.497708,-0.018193
AGCAAACTGGGGTCGCCGCG,-5.156543,-4.666824,6.330266,-0.163140,-4.111705,-3.197671,-2.532976,-3.621127,-5.541826,-1.903153,...,-0.810202,0.187660,-0.399920,0.099022,0.158685,-0.493431,-0.709226,-0.009102,-0.555709,0.100347
CCAACTTCTACGCAGAACAG,-3.001452,-1.952793,7.175604,1.029386,-5.109564,-1.148981,-3.098513,-3.025841,-4.434229,-1.517011,...,-0.734123,0.459329,0.483114,0.514906,-0.035883,0.473481,-0.380109,0.027882,-0.243744,0.546369
GCTGAGCCAGTTCAGAAGCA,-4.364909,-8.228566,7.319953,-2.987663,-6.204640,0.652150,-0.803073,1.410967,-5.715381,-1.810164,...,-0.457922,0.390293,-0.127588,0.276622,0.287196,0.645706,-0.070643,0.322795,-0.574925,0.361378
ACCTCCGCTCCCAATCTACC,-8.573794,12.348477,1.488141,0.908333,1.742249,-4.127612,2.189292,2.768682,-3.185694,-4.918044,...,-0.329269,-0.982739,-0.727424,0.337344,0.015460,-0.391230,0.643675,-0.084215,-0.505240,-0.360643


In [6]:
df_guide_pca_updated = df_guide_pca.reset_index()
pca_feat_list = list(df_guide_pca_updated.columns)
feat_list = ['Metadata_Foci_Barcode_MatchedTo_GeneCode']
feat_list.extend(pca_feat_list)
df_guide_pca_updated["Metadata_Foci_Barcode_MatchedTo_GeneCode"] = gene_list
df_guide_pca_updated = df_guide_pca_updated[feat_list]
df_guide_pca_updated

,Metadata_Foci_Barcode_MatchedTo_GeneCode,Metadata_Foci_Barcode_MatchedTo_Barcode,0,1,2,3,4,5,6,7,...,117,118,119,120,121,122,123,124,125,126
0,AARS2,AAAGGCGGCCCTCACGGCCG,-5.785421,-0.378360,11.583177,1.795391,-4.140418,1.017659,-0.860803,-1.168504,...,0.221074,-0.369346,-0.458958,0.122728,0.344767,0.680401,-0.462260,-0.009803,-0.497708,-0.018193
1,AARS2,AGCAAACTGGGGTCGCCGCG,-5.156543,-4.666824,6.330266,-0.163140,-4.111705,-3.197671,-2.532976,-3.621127,...,-0.810202,0.187660,-0.399920,0.099022,0.158685,-0.493431,-0.709226,-0.009102,-0.555709,0.100347
2,AARS2,CCAACTTCTACGCAGAACAG,-3.001452,-1.952793,7.175604,1.029386,-5.109564,-1.148981,-3.098513,-3.025841,...,-0.734123,0.459329,0.483114,0.514906,-0.035883,0.473481,-0.380109,0.027882,-0.243744,0.546369
3,AARS2,GCTGAGCCAGTTCAGAAGCA,-4.364909,-8.228566,7.319953,-2.987663,-6.204640,0.652150,-0.803073,1.410967,...,-0.457922,0.390293,-0.127588,0.276622,0.287196,0.645706,-0.070643,0.322795,-0.574925,0.361378
4,AARSD1,ACCTCCGCTCCCAATCTACC,-8.573794,12.348477,1.488141,0.908333,1.742249,-4.127612,2.189292,2.768682,...,-0.329269,-0.982739,-0.727424,0.337344,0.015460,-0.391230,0.643675,-0.084215,-0.505240,-0.360643
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2395,nontargeting,TAAGATCCGCGGGTGGCAAC,3.594452,16.721920,4.692836,15.144797,3.298360,6.652650,5.005860,-2.686711,...,1.526243,-0.532978,1.125093,-0.571523,0.194595,0.728165,-0.780570,0.854488,0.263233,1.096793
2396,nontargeting,TCCCGGTTGGTGAACGATAC,-3.707674,-1.051954,8.007314,2.893375,-3.830191,-1.023149,-0.019260,-4.908410,...,-0.874737,0.178808,-0.059155,0.292091,0.430619,0.419389,-0.746553,-0.109805,0.644921,-0.068344
2397,nontargeting,TGCCGTGAAAAGACGCTGCG,-4.392745,-0.656837,9.132958,3.243540,-0.326912,-1.492637,1.460434,-2.069938,...,-0.066345,-0.184594,-0.794681,0.099402,-0.243688,-0.286918,-0.030739,0.165273,-0.316185,-0.370101
2398,nontargeting,TGGCCACGAATTCCGCCGCC,-5.280626,-2.771174,7.173799,4.407507,-1.847060,0.900133,-1.079966,-2.242192,...,-0.549242,-0.250601,-0.292704,-0.156792,0.354184,0.211968,-0.465863,0.272012,-0.167375,0.375059


In [7]:
def calculate_mean_similarity(cosine_array):
    total_sum = 0
    for i in range(4):
        similarities = cosine_array[i]
        target_sum = 0
        for j in range(4):
            target_sum += similarities[j]  
        target_mean = ((target_sum-1)/3)
        total_sum += target_mean

    total_mean = total_sum / 4
    return total_mean

hit_guide_sim_list =[]
for i in range(len(whole_cell_hit_list)):
    df_temp = df_guide_pca_updated.query("Metadata_Foci_Barcode_MatchedTo_GeneCode == @whole_cell_hit_list[@i]")
    df_temp = df_temp.drop(['Metadata_Foci_Barcode_MatchedTo_Barcode'],axis=1)
    df_temp = df_temp.set_index("Metadata_Foci_Barcode_MatchedTo_GeneCode")
    cosine_array = cosine_similarity(df_temp)
    hit_guide_sim = calculate_mean_similarity(cosine_array)
    hit_guide_sim_list.append(hit_guide_sim)
average_cosine_distance = sum(hit_guide_sim_list)/len(hit_guide_sim_list)
print('Average cosine distance: ',average_cosine_distance)

Average cosine distance:  0.5766806250215711


In [9]:
def cosine_to_df(df_temp, cosine_array, i):
    cosine_list = cosine_array[i]
    gene_list = list(df_temp.index)
    cosine_df = pd.DataFrame(index=gene_list)
    cosine_df['cosine'] = cosine_list
    cosine_df = cosine_df.sort_values('cosine',ascending=False)   
    return cosine_df

def ap_from_cosine_df(cosine_df,gene,n=10):    
    #print(cosine_df.iloc[:20])
    index_list = list(cosine_df.index)
    boolean = [1 if  i == gene else 0 for i in index_list ]
    grades_list=[]
    for i in range(2,n+2):
        pre_grade = sum(boolean[1:i])/(i-1)
        grades_list.append(pre_grade*boolean[i-1])
    return sum(grades_list)/3

def calculate_map(df_guide, gene):
    df_temp = df_guide.query("Metadata_Foci_Barcode_MatchedTo_GeneCode == 'nontargeting' | Metadata_Foci_Barcode_MatchedTo_GeneCode == @gene")
    df_temp = df_temp.drop(['Metadata_Foci_Barcode_MatchedTo_Barcode'],axis=1)
    df_temp = df_temp.set_index("Metadata_Foci_Barcode_MatchedTo_GeneCode")
    #print(df_temp)
    ap_list = []
    cosine_array = cosine_similarity(df_temp)
    for guide in range(4):
        cosine_df = cosine_to_df(df_temp, cosine_array, guide)
        #print(cosine_df[:10])
        guide_ap = ap_from_cosine_df(cosine_df,gene,10)
        ap_list.append(guide_ap)
    return np.mean(ap_list)

In [10]:
genes_list = all_genes_list
map_list = []
for i in range(len(genes_list)):
    gene = genes_list[i]
    #print(f"Calculating mean average precision for gene: {gene}")
    gene_map = calculate_map(df_guide_pca_updated, gene)
    #map_list.append([gene, gene_map])
    map_list.append(gene_map)
print(f'For all genes ({len(all_genes_list)} genes) the mAP values is',np.mean(map_list))

For all genes (590 genes) the mAP values is 0.4684997982243745


In [11]:
genes_list = whole_cell_hit_list
map_list = []
for i in range(len(genes_list)):
    gene = genes_list[i]
    #print(f"Calculating mean average precision for gene: {gene}")
    gene_map = calculate_map(df_guide_pca_updated, gene)
    #map_list.append([gene, gene_map])
    map_list.append(gene_map)
print(f'For whole cell hits ({len(whole_cell_hit_list)} genes) the mAP values is',np.mean(map_list))

For whole cell hits (294 genes) the mAP values is 0.716889599755246


In [12]:
genes_list = comp_spec_hit_list
map_list = []
for i in range(len(genes_list)):
    gene = genes_list[i]
    #print(f"Calculating mean average precision for gene: {gene}")
    gene_map = calculate_map(df_guide_pca_updated, gene)
    #map_list.append([gene, gene_map])
    map_list.append(gene_map)
print(f'For compartment hits ({len(comp_spec_hit_list)} genes) the mAP values is',np.mean(map_list))

For compartment hits (94 genes) the mAP values is 0.3203130276933468


In [13]:
genes_list = non_hit_list
map_list = []
for i in range(len(genes_list)):
    gene = genes_list[i]
    #print(f"Calculating mean average precision for gene: {gene}")
    gene_map = calculate_map(df_guide_pca_updated, gene)
    #map_list.append([gene, gene_map])
    map_list.append(gene_map)
print(f'For non hits ({len(non_hit_list)} genes) the mAP values is',np.mean(map_list))

For non hits (202 genes) the mAP values is 0.17594016842160407
